## T1 Feature Engineering

### 1. Load Packages and Set Paths

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

# Ignore warnings from pandas
warnings.filterwarnings('ignore')

# --- 📂 Step 1: Set File Paths ---
data_dir = "../DataSet/"
output_dir = "./"

# Ensure output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"📂 Data source directory: {data_dir}")

📂 Data source directory: ../DataSet/


### 2. Read Raw Data

In [2]:
if not df_txn.empty:
    print("⚙️ Starting T1 feature engineering...\n")

    # Get all unique accounts
    all_accts = pd.concat([df_txn['from_acct'], df_txn['to_acct']]).unique()
    df_t1_features = pd.DataFrame({'acct': all_accts})

    # T1.1: Number of unique destination accounts transferred to
    print("   - Calculating number of unique destination accounts...\n")
    df_num_to = df_txn.groupby('from_acct')['to_acct'].nunique().reset_index()
    df_num_to.columns = ['acct', 'num_unique_dest_accts']
    df_t1_features = pd.merge(df_t1_features, df_num_to, on='acct', how='left')

    # T1.2: Number of unique source accounts received from
    print("   - Calculating number of unique source accounts...\n")
    df_num_from = df_txn.groupby('to_acct')['from_acct'].nunique().reset_index()
    df_num_from.columns = ['acct', 'num_unique_source_accts']
    df_t1_features = pd.merge(df_t1_features, df_num_from, on='acct', how='left')

    # T1.3: High frequency transactions
    print("   - Identifying high-frequency transaction accounts...\n")
    df_from_counts = df_txn.groupby('from_acct').size().reset_index(name='send_count')
    df_to_counts = df_txn.groupby('to_acct').size().reset_index(name='recv_count')
    df_counts = pd.merge(df_from_counts, df_to_counts, left_on='from_acct', right_on='to_acct', how='outer').fillna(0)
    df_counts['acct'] = df_counts['from_acct'].where(df_counts['from_acct'].notna(), df_counts['to_acct'])
    df_counts['total_txns'] = df_counts['send_count'] + df_counts['recv_count']
    high_freq_threshold = df_counts['total_txns'].quantile(0.95)
    print(f"   - High frequency threshold (95th percentile): {high_freq_threshold}\n")
    df_counts['is_high_freq'] = (df_counts['total_txns'] > high_freq_threshold).astype(int)
    df_t1_features = pd.merge(df_t1_features, df_counts[['acct', 'is_high_freq']], on='acct', how='left')

    # T1.4: 是否有外幣 (has_foreign_currency)
    # 根據使用者提供的資訊，資料表說明中提及 currency_type 代表幣別，判斷是否為 TWD 即可。
    print("   - Identifying accounts with foreign currency transactions based on 'currency_type'...\n")
    
    # Assuming 'currency_type' column exists in df_txn based on user's input
    # Check if 'currency_type' column actually exists before using it
    if 'currency_type' in df_txn.columns:
        df_txn['is_foreign_currency_txn'] = (df_txn['currency_type'] != 'TWD').astype(int)

        # An account has foreign currency if any of its transactions involve foreign currency
        # Check for 'from_acct'
        foreign_from_accts = df_txn.groupby('from_acct')['is_foreign_currency_txn'].max().reset_index()
        foreign_from_accts.columns = ['acct', 'has_foreign_currency_from']

        # Check for 'to_acct'
        foreign_to_accts = df_txn.groupby('to_acct')['is_foreign_currency_txn'].max().reset_index()
        foreign_to_accts.columns = ['acct', 'has_foreign_currency_to']

        # Merge and combine
        df_foreign_currency = pd.merge(foreign_from_accts, foreign_to_accts, on='acct', how='outer')
        df_foreign_currency['has_foreign_currency'] = (
            df_foreign_currency['has_foreign_currency_from'].fillna(0) +
            df_foreign_currency['has_foreign_currency_to'].fillna(0) > 0
        ).astype(int)

        df_t1_features = pd.merge(df_t1_features, df_foreign_currency[['acct', 'has_foreign_currency']], on='acct', how='left')
    else:
        print("   - Warning: 'currency_type' column not found in transaction data. 'has_foreign_currency' feature cannot be calculated accurately.\n   - Defaulting 'has_foreign_currency' to 0 for all accounts.")
        df_t1_features['has_foreign_currency'] = 0

    # Fill NaN values with 0
    df_t1_features = df_t1_features.fillna(0)

    print("✅ T1 feature engineering complete!\n")
    df_t1_features.head()

⏳ Reading data...
✅ Data reading complete!
   - Transaction records: 4435890 rows
   - Alert accounts: 1004 rows
   - Accounts to be predicted: 4780 rows


### 3. Feature Engineering (T1)

In [3]:
if not df_txn.empty:
    print("⚙️ Starting T1 feature engineering...")

    # Get all unique accounts
    all_accts = pd.concat([df_txn['from_acct'], df_txn['to_acct']]).unique()
    df_t1_features = pd.DataFrame({'acct': all_accts})

    # T1.1: Number of unique destination accounts transferred to
    print("   - Calculating number of unique destination accounts...")
    df_num_to = df_txn.groupby('from_acct')['to_acct'].nunique().reset_index()
    df_num_to.columns = ['acct', 'num_unique_dest_accts']
    df_t1_features = pd.merge(df_t1_features, df_num_to, on='acct', how='left')

    # T1.2: Number of unique source accounts received from
    print("   - Calculating number of unique source accounts...")
    df_num_from = df_txn.groupby('to_acct')['from_acct'].nunique().reset_index()
    df_num_from.columns = ['acct', 'num_unique_source_accts']
    df_t1_features = pd.merge(df_t1_features, df_num_from, on='acct', how='left')

    # T1.3: High frequency transactions
    print("   - Identifying high-frequency transaction accounts...")
    df_from_counts = df_txn.groupby('from_acct').size().reset_index(name='send_count')
    df_to_counts = df_txn.groupby('to_acct').size().reset_index(name='recv_count')
    df_counts = pd.merge(df_from_counts, df_to_counts, left_on='from_acct', right_on='to_acct', how='outer').fillna(0)
    df_counts['acct'] = df_counts['from_acct'].where(df_counts['from_acct'] != 0, df_counts['to_acct'])
    df_counts['total_txns'] = df_counts['send_count'] + df_counts['recv_count']
    high_freq_threshold = df_counts['total_txns'].quantile(0.95)
    print(f"   - High frequency threshold (95th percentile): {high_freq_threshold}")
    df_counts['is_high_freq'] = (df_counts['total_txns'] > high_freq_threshold).astype(int)
    df_t1_features = pd.merge(df_t1_features, df_counts[['acct', 'is_high_freq']], on='acct', how='left')

    # T1.4: Foreign currency transactions (using non-E.Sun bank as a proxy)
    print("   - Identifying accounts with non-E.Sun transactions (proxy for foreign currency)...")
    df_txn['has_non_esun_from'] = (df_txn['from_acct_type'] != '01').astype(int)
    df_txn['has_non_esun_to'] = (df_txn['to_acct_type'] != '01').astype(int)
    df_non_esun_from = df_txn.groupby('from_acct')['has_non_esun_from'].max().reset_index()
    df_non_esun_to = df_txn.groupby('to_acct')['has_non_esun_to'].max().reset_index()
    df_non_esun = pd.merge(df_non_esun_from, df_non_esun_to, left_on='from_acct', right_on='to_acct', how='outer')
    df_non_esun['acct'] = df_non_esun['from_acct'].fillna(df_non_esun['to_acct'])
    df_non_esun['has_foreign_currency'] = (df_non_esun['has_non_esun_from'].fillna(0) + df_non_esun['has_non_esun_to'].fillna(0) > 0).astype(int)
    df_t1_features = pd.merge(df_t1_features, df_non_esun[['acct', 'has_foreign_currency']], on='acct', how='left')

    # Fill NaN values with 0
    df_t1_features = df_t1_features.fillna(0)

    print("✅ T1 feature engineering complete!")
    df_t1_features.head()

⚙️ Starting T1 feature engineering...
   - Calculating number of unique destination accounts...
   - Calculating number of unique source accounts...
   - Identifying high-frequency transaction accounts...
   - High frequency threshold (95th percentile): 12.0
   - Identifying accounts with non-E.Sun transactions (proxy for foreign currency)...
✅ T1 feature engineering complete!


### 4. Create Train and Test Sets

In [4]:
if 'df_t1_features' in locals():
    print("📦 Splitting into train and test sets...")

    # Add labels to features
    df_t1_features_labeled = pd.merge(df_t1_features, df_alert, on='acct', how='left')
    df_t1_features_labeled['label'] = df_t1_features_labeled['acct'].isin(df_alert['acct']).astype(int)

    predict_acct_set = set(df_predict['acct_id'])

    # Test set
    df_test_t1 = df_t1_features_labeled[df_t1_features_labeled['acct'].isin(predict_acct_set)].copy()
    df_test_t1 = df_test_t1.drop(columns=['label'])

    # Train set
    df_train_t1 = df_t1_features_labeled[~df_t1_features_labeled['acct'].isin(predict_acct_set)].copy()

    print(f"✅ Splitting complete! Train: {df_train_t1.shape}, Test: {df_test_t1.shape}")

📦 Splitting into train and test sets...
✅ Splitting complete! Train: (1795326, 7), Test: (4780, 6)


### 5. Save Processed Files

In [5]:
if 'df_train_t1' in locals() and 'df_test_t1' in locals():
    print("💾 Saving files...")
    train_path = os.path.join(output_dir, 'train_features_t1.csv')
    test_path = os.path.join(output_dir, 'test_features_t1.csv')

    df_train_t1.to_csv(train_path, index=False)
    df_test_t1.to_csv(test_path, index=False)

    print(f"   - Train features saved to: {train_path}")
    print(f"   - Test features saved to: {test_path}")
    print("🎉 All steps completed successfully!")

💾 Saving files...
   - Train features saved to: ./train_features_t1.csv
   - Test features saved to: ./test_features_t1.csv
🎉 All steps completed successfully!
